In [ ]:
from PIL import Image
import json
import pandas as pd
import os

# Load JSON data from the uploaded image description
with open('/mnt/data/image.png', 'r') as file:
    data = file.read()

# Convert string data to JSON
json_data = json.loads(data)

# Function to crop images based on bounding box and save them with labels
def crop_and_label_images(json_data, base_path='/mnt/data/'):
    # List to hold image name and label for CSV
    image_label_list = []
    
    # Load the image
    img_path = os.path.join(base_path, json_data['images'][0]['file_name'])
    with Image.open(img_path) as img:
        # Process each annotation (bbox and label)
        for annotation in json_data['annotations']:
            # Extract bbox coordinates
            bbox = annotation['bbox']
            left, top, right, bottom = bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]
            
            # Crop the image
            cropped_img = img.crop((left, top, right, bottom))
            
            # Generate new image name
            cropped_img_name = f"{annotation['id']}_{json_data['images'][0]['file_name']}"
            cropped_img_path = os.path.join(base_path, cropped_img_name)
            
            # Save the cropped image
            cropped_img.save(cropped_img_path)
            
            # Append the image name and label to the list
            image_label_list.append([cropped_img_name, annotation['text']])
    
    # Convert list to DataFrame
    df = pd.DataFrame(image_label_list, columns=['image_name', 'label'])
    
    # Save DataFrame to CSV
    csv_path = os.path.join(base_path, 'labels.csv')
    df.to_csv(csv_path, index=False)
    
    return csv_path

# Call the function and create the CSV
csv_file_path = crop_and_label_images(json_data)

csv_file_path
